# Forecast

## Setup

In [ ]:
import sys
sys.path.append("..")

# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy as sp

from joblib import Parallel, delayed
from itertools import product
from tqdm import tqdm
from pathlib import Path

# Library settings
pd.options.display.max_columns = 999
plt.rcParams["figure.figsize"] = (16, 4)


## Prepare data

In [ ]:
sales = pd.read_csv("../data/sales_train_evaluation.csv")
sales.head()


In [ ]:
calendar = pd.read_csv("../data/calendar.csv", parse_dates=["date"])
calendar.head()

In [ ]:
sell_prices = pd.read_csv("../data/sell_prices.csv")
sell_prices.head()

In [ ]:
def prepare_data(sales, calendar, sell_prices):
    hierarchy_df = sales.iloc[:, 1:6].copy()
    df = sales.drop(columns=["id", "dept_id", "cat_id", "state_id"])
    df = df.reset_index().rename(columns={"index":"unique_id"})
    df_long = pd.melt(df, id_vars=df.columns[:3], var_name="d", value_name="sales")
    df_long = df_long.merge(calendar[["d", "wm_yr_wk"]], on="d", how="left")
    df_long = df_long.merge(sell_prices, on=["item_id", "store_id", "wm_yr_wk"], how="left")
    df_long["dollar_sales"] = df_long["sales"] * df_long["sell_price"]
    df_long.loc[df_long.sales == 0, "dollar_sales"] = 0
    df_long["d"] = df_long["d"].str[2:].astype("int")
    df = pd.pivot_table(df_long, values=["sales", "dollar_sales"], index=["d"], columns=["unique_id"])
    sales_arr = df["sales"].to_numpy().T
    dollar_sales_arr = df["dollar_sales"].to_numpy().T

    filepath = Path("../data/processed/bottom_sales_arr.npy")
    np.save(filepath, sales_arr)
    print(f"Data written to {filepath}")
    
    filepath = Path("../data/processed/bottom_dollar_sales_arr.npy")
    np.save(filepath, dollar_sales_arr)
    print(f"Data written to {filepath}")

    filepath = Path("../data/processed/hierarchy.csv")
    hierarchy_df.to_csv(filepath, index=False)
    print(f"Data written to {filepath}")
    
    return sales_arr, dollar_sales_arr, hierarchy_df

In [ ]:
bottom_sales_arr, bottom_dollar_sales_arr, hierarchy_df = prepare_data(sales, calendar, sell_prices)

## Build S matrix

In [ ]:
hierarchy_df = pd.read_csv("../data/processed/hierarchy.csv")
hierarchy_df.head()

In [ ]:
agg_levels = (
    tuple(["TOTAL"]),
    tuple(['state_id']),
    tuple(['store_id']),
    tuple(['cat_id']),
    tuple(['dept_id']),
    tuple(['state_id', 'cat_id']),
    tuple(['state_id', 'dept_id']),
    tuple(['store_id', 'cat_id']),
    tuple(['store_id', 'dept_id']),
    tuple(['item_id']),
    tuple(['item_id', 'state_id']),
    tuple(['item_id', 'store_id']),
)

In [ ]:
def generate_Smatrix(hierarchy_df, agg_levels, sparse=True):
    arr = np.array
    eye = np.eye
    stack = np.row_stack
    
    if sparse:
        arr = sp.sparse.csr_array
        eye = sp.sparse.eye
        stack = sp.sparse.vstack

    def build_row(hierarchy_df, level, comb):
        row = (hierarchy_df.loc[:, level] == comb).apply(lambda x: all(x), axis=1).astype("int16")
        return row
    
    top_row = np.ones(len(hierarchy_df))
    S_rows = [top_row]

    for level in agg_levels[1:-1]:
        print(level)
        combinations = hierarchy_df.loc[:, level].drop_duplicates().to_numpy()
        rows = Parallel(n_jobs=8)(
            delayed(build_row)(hierarchy_df, level, comb) for comb in combinations
        )
        S_rows.extend(rows)

    S_top = arr(S_rows)
    S_bottom = eye(len(hierarchy_df))
    S = stack([S_top, S_bottom])
    
    filepath = Path("../data/processed/S.npz")
    sp.sparse.save_npz(filepath, S)
    print(f"Data written to {filepath}")
    return S

In [ ]:
S = generate_Smatrix(hierarchy_df, agg_levels)

## Generate all the timeseries from the hierarchy

In [ ]:
bottom_sales_arr = np.load("../data/processed/bottom_sales_arr.npy")
bottom_dollar_sales_arr = np.load("../data/processed/bottom_dollar_sales_arr.npy")
S = sp.sparse.load_npz("../data/processed/S.npz")

In [ ]:
def generate_Ymatrix(S, bottom_sales_arr, bottom_dollar_sales_arr):
    sales_arr = S @ bottom_sales_arr
    dollar_sales_arr = S @ bottom_dollar_sales_arr

    filepath = Path("../data/processed/sales_arr.npy")
    np.save(filepath, sales_arr)
    print(f"Data written to {filepath}")

    filepath = Path("../data/processed/dollar_sales_arr.npy")
    np.save(filepath, dollar_sales_arr)
    print(f"Data written to {filepath}")

    return sales_arr, dollar_sales_arr

In [ ]:
sales_arr, dollar_sales_arr = generate_Ymatrix(S, bottom_sales_arr, bottom_dollar_sales_arr)

## Add calendar features

In [ ]:
sales_arr = np.load("../data/processed/sales_arr.npy")
dollar_sales_arr = np.load("../data/processed/dollar_sales_arr.npy")
calendar = pd.read_csv("../data/calendar.csv", parse_dates=["date"])

In [ ]:
def factorize_calendar(calendar):
    calendar["event_name_1"] = pd.factorize(calendar["event_name_1"])[0]
    calendar["event_type_1"] = pd.factorize(calendar["event_type_1"])[0]
    calendar["event_name_2"] = pd.factorize(calendar["event_name_2"])[0]
    calendar["event_type_2"] = pd.factorize(calendar["event_type_2"])[0]
    return calendar

In [ ]:
def dummify_calendar(calendar):
    calendar = pd.get_dummies(
        calendar, 
        columns=[
            "wday",
            "month",
            "year",
        ], 
        drop_first=True
    )
    calendar = pd.get_dummies(
        calendar, 
        columns=[
            "event_name_1", 
            "event_type_1", 
            "event_name_2", 
            "event_type_2",
        ], 
        drop_first=False
    )
    return calendar

In [ ]:
def clean_calendar(calendar, dummies=False):
    calendar = calendar.drop(columns=["wm_yr_wk", "weekday"])
    calendar = calendar.rename(columns={"date": "ds"})
    calendar["d"] = calendar["d"].str[2:].astype("int")
    if dummies:
        calendar = dummify_calendar(calendar)
    else:
        calendar = factorize_calendar(calendar)
    return calendar

In [ ]:
calendar_dummies = clean_calendar(calendar, dummies=True)
calendar_categorical = clean_calendar(calendar, dummies=False)

In [ ]:
calendar_dummies.head()

In [ ]:
calendar_categorical.head()

In [ ]:
def generate_panel_df(sales_arr, regressors_df=None):
    yx_df_list = []
    for i in tqdm(range(sales_arr.shape[0])):
        y = sales_arr[i, :]
        yx_df = pd.DataFrame(y, index=range(len(y)), columns=["y"])
        if regressors_df is not None:
            yx_df = pd.merge(yx_df, regressors_df, left_index=True, right_index=True, how="left")
        yx_df.insert(0, "unique_id", i)
        yx_df_list.append(yx_df)
    panel_df = pd.concat(yx_df_list)

    filepath = Path("../data/processed/panel_df.parquet")
    panel_df.to_parquet(filepath)
    print(f"Data written to {filepath}")

    return panel_df

In [ ]:
panel_df = generate_panel_df(sales_arr, calendar_categorical)

In [ ]:
panel_df.head()

## Forecast

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, ETS

In [ ]:
panel_df = pd.read_parquet("../data/processed/panel_df.parquet")


In [ ]:
panel_df = panel_df.iloc[:, :3]

In [ ]:
models = [
    ETS(season_length=7, model='ZZA'),
    AutoARIMA()
]

model = StatsForecast(
    df=panel_df, 
    models=models,
    freq='D', 
    n_jobs=-1,
)

fcst_df = model.forecast(28).reset_index()
fcst_df.head()

In [ ]:
filepath = "../fcst/fcst_df.csv"
fcst_df.to_csv(filepath)
print(f"Data written to {filepath}")

In [ ]:
i = 10000
Y_hat_df.loc[Y_hat_df.unique_id == i].set_index("ds")["ETS"].plot()
Y_test_df.loc[Y_test_df.unique_id == i].set_index("ds")["y"].plot()